# Digital Badges for First Responders: Real-World Data and Preliminary Workflow Plans

**SBIR Phase I Contract #70RSAT23C00000021**  
**Merit International, Inc.**  
**2023-09-16**

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;width:75%;margin:0 auto;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
</style>
<table class="tg">
<tbody>
  <tr>
    <td class="tg-0pky">Title</td>
    <td class="tg-0pky">Department of Homeland Security (DHS)- Small Business Innovation Research (SBIR) - First Responders Digital Badges (Topic # 23.1 DHS231-003-I)</td>
    <td class="tg-0pky" rowspan="6">This is SBIR/STTR Data (or is Computer Software or a Prototype that embodies or includes SBIR/STTR data) to which the Awardee has SBIR/STTR Data Rights and to which the Federal Government has received SBIR/STTR Technical Data RIghts (or SBIR/STTR Computer Software Rights) during the SBIR/STTR Protection Period and rights of use for Government Purposes after the SBIR/STTR Protection Period, as those terms are defined in the SBIR/STTR Funding Agreement. Awards issued by the U.S. Department of Energy are subject to Unlimited Rights after the SBIR/STTR Protection Period, as that term is defined in the SBIR/STTR Funding Agreement. Any reproduction of the SBIR/STTR Data or portions of such data marked with this legend must also reproduce the markings.</td>
  </tr>
  <tr>
    <td class="tg-0pky">Funding Agreement</td>
    <td class="tg-0pky">70RSAT23C00000021</td>
  </tr>
  <tr>
    <td class="tg-0pky">Award Date</td>
    <td class="tg-0pky">08 May 2023</td>
  </tr>
  <tr>
    <td class="tg-0pky">SBIR Protection Period</td>
    <td class="tg-0pky">08 May 2043</td>
  </tr>
  <tr>
    <td class="tg-0pky">SBIR Awardee</td>
    <td class="tg-0pky">Merit International, Inc.</td>
  </tr>
  <tr>
    <td class="tg-0pky">SBIR Awardee Address</td>
    <td class="tg-0pky">100 S Murphy Ave. Suite 200 Sunnyvale, CA 94086</td>
  </tr>
</tbody>
</table>

## Overview

This report describes a proof-of-concept that demonstrates technical feasibility for the support of interoperability and location awareness in an extended version of the Merit verified identity platform. It was developed as a Python notebook with executable code, and is meant to present two key findings of the SBIR Phase I effort:

1. Merit credentials can be straightforwardly transformed into [W3C Verifiable Credentials Data Model v2.0](https://www.w3.org/TR/vc-data-model-2.0/) conformant documents, which can then be signed, issued and verified by Merit and other third parties in the W3C verifiable credentials ecosystem.

1. There is a feasible and innovative approach possible towards extending verifiable credentials generated from Merit credentials with geofencing and geolocation information to provide location awareness in the verification process in support of use cases in applications by first responder organizations.

In the discussion below, we will describe the data used to demonstrate the above findings, and provide recommendations as to how a follow-on project would address the commercialization of our approach. We will also indicate where design choices were taken to constrain the amount of effort in the development of the proof of concept, but it is worth emphasizing that what we present below is, to the best of our knowledge, fully conformant with the W3C Verifiable Credentials Data Model v2.0 specification, as well as the rest of the stakeholder needs detailed in the project’s Statement of Work. 

This document therefore constitutes the deliverable for milestone 4 of the project, the goal of which was to “demonstrate technology feasibility based on output from milestone 3, and provide a report with a summary, data analysis, findings, and recommendations.”

We first import a handful of standard Python packages that we will use below to support serialization, deserialization, validation, signing and verification of credentials.

In [1]:
import json, jsonschema, jwt, datetime, uuid, requests

## Preliminaries

We now define a few functions that will provide support for the signing and verification of verifiable credentials.

### `urn_uuid`: a helper function to mint URN UUID IRIs to use as identifiers

In this proof of concept, we use URN UUID IRIs as identifiers. Commercialization may in addition use dereferencable HTTP(S) URLs or decentralized identifier (DID) URLs instead, depending on application requirements as determined in consultation with customers and other in the W3C Verifiable Credentials ecosystem.

In [2]:
def urn_uuid(uuid):
    return f'urn:uuid:{uuid}'

### `MockKeyManager`: a mock key manager to support demonstration of verification of signed credentials and presentations

For the simplicity of this proof of concept, we are using symmetric keys to sign, encode, and decode verifiable credentials. Commercialization will require the implementation of a industrial-strength approach using a robust public key infrastructure (such as is already deployed for use with the current Merit platform), and client applications that can cache public keys to support "no-phone-home" use of credentials in situations, as often encountered in emergency responder work, that have limited or no wireless network access.

In [3]:
class MockKeyManager:

    def __init__(self):
        self.secrets = {}

    def add(self, key):
        kid = urn_uuid(uuid.uuid4())
        self.secrets[kid] = key
        return kid

    def get(self, kid):
        if kid in self.secrets:
            return self.secrets[kid]
        else:
            return None

    def delete(self, kid):
        if kid in self.secrets:
            del self.secrets[kid]
        else:
            return None

    def reset(self):
        self.secrets = {}

    def list(self):
        for kid in self.secrets:
            print(f'{kid}: {self.secrets[kid]}')

### `sign`: signing and encrypting a plaintext credential or presentation

In this proof of concept, without loss of generality we will use external proofs via JSON Web Tokens (JWTs), encrypted as mentioned above using symmetric keys. While this is comformant with the W3C Verifiable Credentials Data Model v2.0 specification, commercialization will almost certainly involve the use of other mechanisms, including internal proofs and asymmetric public key infrastructure, depending on application requirements.

In [4]:
def sign(key_manager, credential, key, verifier):
    kid = key_manager.add(key)
    headers = {
    "alg": "HS256",
    "typ": "JWT",
    "kid": kid
    }
    payload = {
        "iss": credential["issuer"],  # Issuer
        "aud": verifier,  # Audience
        "exp": datetime.datetime.utcnow() + datetime.timedelta(days=1),  # Expiration time
        "nbf": datetime.datetime.utcnow(),  # Not before
        "iat": datetime.datetime.utcnow(),  # Issued at
        "kid": kid,
    }
    if "VerifiableCredential" in credential["type"]:
        payload["sub"] = credential["credentialSubject"]["id"]
        payload["vc"] = credential
    elif "VerifiablePresentation" in credential["type"]:
        payload["vp"] = credential
    return jwt.encode(payload, key, algorithm='HS256', headers=headers)

### `verify`: verifying and decrypting a signed credential or presentation

We provide a way of verifying and decrypting W3C Verifiable Credentials Data Model v2.0 conformant documents that handles both verifiable credentials and verifiable presentations using the same function.

In [5]:
def verify(key_manager, signed_credential, verifier):
    try:
        key = key_manager.get(jwt.get_unverified_header(signed_credential)["kid"])
        decoded_payload = jwt.decode(signed_credential, key, algorithms='HS256', audience=verifier)
        if 'vp' in decoded_payload:
            decoded_payload["vp"]["verifiableCredential"] = [ 
                verify(key_manager, payload, verifier) for payload in decoded_payload["vp"]["verifiableCredential"] 
            ]
            return decoded_payload['vp']
        else:
            return decoded_payload['vc']
    except Exception as e:
        print(f'Verification error: {e}')
        return None

### Global variables 

We define a couple of global variables, one for the key manager we will use in the demonstration, and one for the verifier we will use as the audience for the credentials and presentation.

In [6]:
KEY_MANAGER = MockKeyManager()
VERIFIER = "https://verifier.example.gov"

## Extending the Merit platform to support interoperability using the W3C Verifiable Credentials Data Model

A key objective of the SBIR Phase I project is to show how the Merit platform can support open standards for credentialing in a manner acceptable to the first responder community. Below we use an example Merit credential, produced using the current Merit platform, and show how it can be transformed into a W3C Verifiable Credentials Data Model v2.0 conformant document, without loss of functionality.

### An example Merit credential

We now load a JSON serialization of a Merit credential from disk. This "merit" is an example taken from one of our quality assurance test suites. It represents that a particular individual has administrative rights for a specific organization's credential issuance system. The merit contains basic credential metadata that maps relatively straightforwardly to equivalents in the W3C Verifiable Credential Data Model v2.0; the `fields` section of the JSON serialization contains what is equivalent to the `credentialSubject` of a verifiable credential, and in additional metadata that provides type and provenance information used by the Merit Organization Portal adminstrative application for administrators to create and manage the definition and issuance of credentials.

In [7]:
merit = json.load(open("test_merit.json", "r"))
merit

{'id': 'dc30880d-2d4b-447c-905e-b79075aea95b',
 'acceptedAt': '2023-07-17T08:52:19.489566Z',
 'active': True,
 'activenessFailures': None,
 'authorizedAt': '2023-07-17T08:52:15.960677Z',
 'completed': True,
 'completenessFailures': None,
 'createdAt': '2023-07-17T15:52:04.3018Z',
 'description': 'Template for admin merit',
 'fields': [{'canShare': False,
   'description': "Use this to store recipient's first name",
   'fieldKind': {'fieldType': 'Text',
    'id': 'a9fa2832-4107-4c95-b7d8-0cdd15a7e2bc',
    'name': 'Text'},
   'name': 'First Name',
   'templateFieldID': '1b397a27-14a0-4d43-a36f-6006611f4073',
   'templateFieldLineage': ['3dc3f769-70cc-4e7e-81cc-ed103f9eb601'],
   'updatedAt': '2023-07-17T15:52:19.496539Z',
   'validationErrors': None,
   'value': 'Hank'},
  {'canShare': False,
   'description': "Use this to store recipient's last name",
   'fieldKind': {'fieldType': 'Text',
    'id': 'a9fa2832-4107-4c95-b7d8-0cdd15a7e2bc',
    'name': 'Text'},
   'name': 'Last Name',
   

### A verifiable credential generated from the example Merit credential

Next, we take the example Merit credential and use the data and metadata contained in it to generate a verifiable container document that is conformant with the W3C Verifiable Credential Data Model v2.0 standard. The mapping is straightforward and valid for any Merit credential, though lossy in the sense that the type and provenance metadata associated with the `fields` section of the Merit credential is elided in generating the `credentialSubject` section of the verifiable credential. However, the key information relative to the nature of the credential for use by the holder and verifier is completely preserved, and the semantics of the metadata is kept intact. The `credentialStatus` section of the verifiable credential provides an HTTPS endpoint for a verifier to access to confirm the status of the credential in real-time, it refers to a property-based extension of the standard's support for expressing credential status in a manner that conforms with the semantics established in the Merit platform; commercialization will require that extension to be defined and registered in the [Verifiable Credentials Specification Directory](https://w3c.github.io/vc-specs-dir/) hosted by the W3C.

In [8]:
id = urn_uuid(merit["id"])
issuer = urn_uuid(merit["issuer"]["id"])
subject = urn_uuid(merit["recipient"]["id"])

verifiable_credential = {
    "@context": [
        "https://www.w3.org/2018/credentials/v2",
        "https://www.w3.org/2018/credentials/examples/v2",
        "https://schema.org/"
    ],
    "issuer": issuer,
    "id": id,
    "type": ["VerifiableCredential", merit["name"]],
    "credentialSubject": {
        "id": subject,
        "merit": { field["name"]: field["value"] for field in merit["fields"] },
    },
    "issuanceDate": merit["createdAt"],
    "credentialStatus": {
        "id": f'https://vc.merit.com/status/{requests.utils.quote(id)}',
        "type": "MeritCredentialStatusList2024",
        "statusPurpose": merit['state']['name'],
        "occurredAt": merit['state']['occurredAt']
    }
}
verifiable_credential

{'@context': ['https://www.w3.org/2018/credentials/v2',
  'https://www.w3.org/2018/credentials/examples/v2',
  'https://schema.org/'],
 'issuer': 'urn:uuid:9a5eb92a-b34a-42b9-8fb7-10ce9449e83e',
 'id': 'urn:uuid:dc30880d-2d4b-447c-905e-b79075aea95b',
 'type': ['VerifiableCredential', 'Admin merit'],
 'credentialSubject': {'id': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
  'merit': {'First Name': 'Hank',
   'Last Name': 'Lindgren',
   'Email': 'oleg+user2@gomerits.com',
   'Issuing Org Name': 'Merit International, Inc.',
   'Org UUID': 'user2',
   'Admin Phone Number': '6317070926',
   'Org Legal Name': 'Org legal name 74d00472-85d5-4c6b-81d3-5b6353a8a7fd',
   'Org Name': 'Organization Name b6e9f27b-6214-4f57-a670-5e02d2a1fdeb',
   'Org Description': 'This is very important organization',
   'Org Governing Country': 'US',
   'Org Governing State': 'CA'}},
 'issuanceDate': '2023-07-17T15:52:04.3018Z',
 'credentialStatus': {'id': 'https://vc.merit.com/status/urn%3Auuid%3Adc30880d-2d

### JSON Schema validation of the verifiable credential

We now define a JSON schema that can validate the encoding of our generated plaintext verifiable credential. Commercialization may entail the elaboration of this schema; alternatively, official schemas provided by the W3C Verifiable Credentials ecosystem may be used instead.

In [9]:
vc_schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "@context": {
            "type": "array",
            "items": {
                "type": "string"
            },
            "minItems": 1
        },
        "id": {
            "type": "string"
        },
        "type": {
            "type": "array",
            "items": {
                "type": "string"
            },
            "contains": {
                "const": "VerifiableCredential"
            }
        },
        "issuer": {
            "type": "string"
        },
        "issuanceDate": {
            "type": "string",
            "format": "date-time"
        },
        "credentialSubject": {
            "type": "object",
            "properties": {
                "id": {
                    "type": "string"
                }
            },
            "required": ["id"]
        },
        "credentialStatus": {
            "type": "object",
            "properties": {
                "id": {
                    "type": "string"
                },
                "type": {
                    "type": "string"
                },
                "statusPurpose": {
                    "type": "string"
                },
                "occurredAt": {
                    "type": "string",
                    "format": "date-time"
                }
            },
            "required": ["id"]
        },
        "proof": {
            "type": "object",
            "properties": {
                "type": {
                    "type": "string"
                },
                "created": {
                    "type": "string",
                    "format": "date-time"
                },
                "proofValue": {
                    "type": "string"
                },
                "verificationMethod": {
                    "type": "string"
                },
                "jws": {
                    "type": "string"
                }
            },
            "required": ["type", "created", "proofValue"]
        }
    },
    "required": ["@context", "id", "type", "issuer", "issuanceDate", "credentialSubject", "credentialStatus"]
}

We now validate the verifiable credential using the schema. 

In [10]:
try:
    jsonschema.validate(instance=verifiable_credential, schema=vc_schema)
    print("Credential is valid.")
except jsonschema.exceptions.ValidationError as e:
    print("Credential is not valid:", e)

Credential is valid.


### JSON Web Token (JWT) serialization of the verifiable credential

Having validated the verifiable credential, we now sign and secure it by generating a JWT token. 

In [11]:
signed_vc = sign(KEY_MANAGER, verifiable_credential, "secret", VERIFIER)
signed_vc

'eyJhbGciOiJIUzI1NiIsImtpZCI6InVybjp1dWlkOjRiYTU1MDFjLWE0MjYtNDNkYy1hNmZjLWY5YjIyY2I5OWQzYSIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJ1cm46dXVpZDo5YTVlYjkyYS1iMzRhLTQyYjktOGZiNy0xMGNlOTQ0OWU4M2UiLCJhdWQiOiJodHRwczovL3ZlcmlmaWVyLmV4YW1wbGUuZ292IiwiZXhwIjoxNjk0OTg5MTYxLCJuYmYiOjE2OTQ5MDI3NjEsImlhdCI6MTY5NDkwMjc2MSwia2lkIjoidXJuOnV1aWQ6NGJhNTUwMWMtYTQyNi00M2RjLWE2ZmMtZjliMjJjYjk5ZDNhIiwic3ViIjoidXJuOnV1aWQ6MTk2YThjOGYtNmZkOS00NTZiLTgxOTMtNDE1OWI4ZGMzYWI2IiwidmMiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjIiLCJodHRwczovL3d3dy53My5vcmcvMjAxOC9jcmVkZW50aWFscy9leGFtcGxlcy92MiIsImh0dHBzOi8vc2NoZW1hLm9yZy8iXSwiaXNzdWVyIjoidXJuOnV1aWQ6OWE1ZWI5MmEtYjM0YS00MmI5LThmYjctMTBjZTk0NDllODNlIiwiaWQiOiJ1cm46dXVpZDpkYzMwODgwZC0yZDRiLTQ0N2MtOTA1ZS1iNzkwNzVhZWE5NWIiLCJ0eXBlIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiQWRtaW4gbWVyaXQiXSwiY3JlZGVudGlhbFN1YmplY3QiOnsiaWQiOiJ1cm46dXVpZDoxOTZhOGM4Zi02ZmQ5LTQ1NmItODE5My00MTU5YjhkYzNhYjYiLCJtZXJpdCI6eyJGaXJzdCBOYW1lIjoiSGFuayIsIkxhc3QgTmFtZSI6IkxpbmRncmVuIiwiRW

Now we reverse the process, showing how, using the key manager to retrieve the key based on the key identifier provided in the header of the token, we can recover a conformant plaintext verifiable credential.

In [12]:
key = KEY_MANAGER.get(jwt.get_unverified_header(signed_vc)["kid"])
verifiable_credential = verify(KEY_MANAGER, signed_vc, VERIFIER)
verifiable_credential

{'@context': ['https://www.w3.org/2018/credentials/v2',
  'https://www.w3.org/2018/credentials/examples/v2',
  'https://schema.org/'],
 'issuer': 'urn:uuid:9a5eb92a-b34a-42b9-8fb7-10ce9449e83e',
 'id': 'urn:uuid:dc30880d-2d4b-447c-905e-b79075aea95b',
 'type': ['VerifiableCredential', 'Admin merit'],
 'credentialSubject': {'id': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
  'merit': {'First Name': 'Hank',
   'Last Name': 'Lindgren',
   'Email': 'oleg+user2@gomerits.com',
   'Issuing Org Name': 'Merit International, Inc.',
   'Org UUID': 'user2',
   'Admin Phone Number': '6317070926',
   'Org Legal Name': 'Org legal name 74d00472-85d5-4c6b-81d3-5b6353a8a7fd',
   'Org Name': 'Organization Name b6e9f27b-6214-4f57-a670-5e02d2a1fdeb',
   'Org Description': 'This is very important organization',
   'Org Governing Country': 'US',
   'Org Governing State': 'CA'}},
 'issuanceDate': '2023-07-17T15:52:04.3018Z',
 'credentialStatus': {'id': 'https://vc.merit.com/status/urn%3Auuid%3Adc30880d-2d

## Extending verifiable credentials to support geo-aware verification

A second key objective of the SBIR Phase I project is to show how geolocation and geofencing capabilities can be layered onto the Merit platform's support for interoperability in a way that address use cases in the first responder application domain, where many scenarios involve the use of digital badging to manage and monitor movement of authorized personnel into specific geographical locations and regions. Below we show how the Merit platform coupled with the interoperability approach demonstrated above can be enhanced to provide such capabilities. Merit personnel first developed a proof of concept demonstrating this in October 2022, using the existing Merit platform and representation of credentials; below, we show how this concept would work, and be made interoperable with other third-party solutions, by leveraging the representational flexibility of the W3C Verifiable Credentials Data Model to provide a preliminary workflow plan for delivering location awareness.

### Adding geofencing into the credentialSubject of the verifiable credential

Given the above verifiable credential that we have generated from a Merit credential, we now show how we can support the incorporation of geofencing data into the `credentialSubject` of the conforming JSON document. For this example, we use the Schema.org vocabulary's `schema:GeoCircle` type and `schema:eligibleRegion` property to limit the credential's scope to the geographical area defined by a circle 50 meters in radius around the location of the auditorium of the General Services Administration Capital Region building at 301 7th St SW, Washington, DC 20024.

In [13]:
verifiable_credential["credentialSubject"]["eligibleRegion"] = {
    "type": "GeoCircle",
    "geoMidpoint": {
        "type": "GeoCoordinates",
        "latitude": "38.88508",
        "longitude": "-77.02295"
    },
    "geoRadius": "50"
}
verifiable_credential

{'@context': ['https://www.w3.org/2018/credentials/v2',
  'https://www.w3.org/2018/credentials/examples/v2',
  'https://schema.org/'],
 'issuer': 'urn:uuid:9a5eb92a-b34a-42b9-8fb7-10ce9449e83e',
 'id': 'urn:uuid:dc30880d-2d4b-447c-905e-b79075aea95b',
 'type': ['VerifiableCredential', 'Admin merit'],
 'credentialSubject': {'id': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
  'merit': {'First Name': 'Hank',
   'Last Name': 'Lindgren',
   'Email': 'oleg+user2@gomerits.com',
   'Issuing Org Name': 'Merit International, Inc.',
   'Org UUID': 'user2',
   'Admin Phone Number': '6317070926',
   'Org Legal Name': 'Org legal name 74d00472-85d5-4c6b-81d3-5b6353a8a7fd',
   'Org Name': 'Organization Name b6e9f27b-6214-4f57-a670-5e02d2a1fdeb',
   'Org Description': 'This is very important organization',
   'Org Governing Country': 'US',
   'Org Governing State': 'CA'},
  'eligibleRegion': {'type': 'GeoCircle',
   'geoMidpoint': {'type': 'GeoCoordinates',
    'latitude': '38.88508',
    'longitud

We sign and secure the newly geofenced verifiable credential by generating another JWT token. 

In [14]:
signed_geofenced_vc = sign(KEY_MANAGER, verifiable_credential, "secret2", VERIFIER)
signed_geofenced_vc

'eyJhbGciOiJIUzI1NiIsImtpZCI6InVybjp1dWlkOjdiZmUyMzFhLTM0MWQtNGQ2MS05NDU5LWIwMWE1MTkzNjk1OSIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJ1cm46dXVpZDo5YTVlYjkyYS1iMzRhLTQyYjktOGZiNy0xMGNlOTQ0OWU4M2UiLCJhdWQiOiJodHRwczovL3ZlcmlmaWVyLmV4YW1wbGUuZ292IiwiZXhwIjoxNjk0OTg5MTYxLCJuYmYiOjE2OTQ5MDI3NjEsImlhdCI6MTY5NDkwMjc2MSwia2lkIjoidXJuOnV1aWQ6N2JmZTIzMWEtMzQxZC00ZDYxLTk0NTktYjAxYTUxOTM2OTU5Iiwic3ViIjoidXJuOnV1aWQ6MTk2YThjOGYtNmZkOS00NTZiLTgxOTMtNDE1OWI4ZGMzYWI2IiwidmMiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjIiLCJodHRwczovL3d3dy53My5vcmcvMjAxOC9jcmVkZW50aWFscy9leGFtcGxlcy92MiIsImh0dHBzOi8vc2NoZW1hLm9yZy8iXSwiaXNzdWVyIjoidXJuOnV1aWQ6OWE1ZWI5MmEtYjM0YS00MmI5LThmYjctMTBjZTk0NDllODNlIiwiaWQiOiJ1cm46dXVpZDpkYzMwODgwZC0yZDRiLTQ0N2MtOTA1ZS1iNzkwNzVhZWE5NWIiLCJ0eXBlIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiQWRtaW4gbWVyaXQiXSwiY3JlZGVudGlhbFN1YmplY3QiOnsiaWQiOiJ1cm46dXVpZDoxOTZhOGM4Zi02ZmQ5LTQ1NmItODE5My00MTU5YjhkYzNhYjYiLCJtZXJpdCI6eyJGaXJzdCBOYW1lIjoiSGFuayIsIkxhc3QgTmFtZSI6IkxpbmRncmVuIiwiRW

### Generating a geolocation verifiable credential

Now that we have a verifiable credential that contains geofencing data, we show how we can additionally provide a verifiable credential that expresses the real-time location of a holder of that credential. The scenario makes several assumptions:

1. The holder's digital wallet is on a mobile device with secure GPS geolocation capabilities.

1. The digital wallet is also capable of generating a verifiable credential, issued by the holder from her mobile device, that encodes real-time geolocation data obtained from the device, signs it, and makes it available to be combined with other verifiable credentials in the wallet to form a verifiable presentation that combines a given credential with the geolocation credential.

Commercialization of this kind of wallet functionality will require development effort to integrate the wallet with device GPS services and to provide the workflow and orchestration that the wallet will need to execute on-board to support this kind of capability. The following scenario does not address how this workflow will be executed in a context-senstive way that does not impact the user experience negatively, particularly given the challenges presented in the sort of operational environments that first responders work in; commercialization efforts will need to pay close attention to make such a system practical. However, our experience with the October 2022 proof-of-concept provides evidence that the technical and usability risks can be effectively managed.

Continuing the above example, we imagine that a holder with a geo-aware digital wallet as described generates a geolocation verifiable credential, which has a `credentialSubject` that uses the Schema.org `schema:GeoCoordinates` type and `schema:geo` property to represent their current location in the food court at 500 L'Enfant Plaza SW, Washington, DC 20024, across the street from the GSA auditorium.

In [15]:
geolocation_vc_id = urn_uuid(uuid.uuid4())
geolocation_vc = {
    '@context': [
        'https://www.w3.org/2018/credentials/v1', 
        'https://www.w3.org/2018/credentials/examples/v1', 
        'https://schema.org/'
    ],
    'issuer': subject,
    'id': geolocation_vc_id,
    'type': ['VerifiableCredential', 'RealtimeGeolocation'],
    'credentialSubject' : {
        'id': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
        'geo': {
            'type': 'GeoCoordinates',
            'latitude': '38.88460',
            'longitude': '-77.02273'
        }
    },
    'issuanceDate': datetime.datetime.utcnow().isoformat() + 'Z'
}
geolocation_vc

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1',
  'https://schema.org/'],
 'issuer': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
 'id': 'urn:uuid:eb5b0800-967f-4d0b-a34a-ae67ecdc9a6b',
 'type': ['VerifiableCredential', 'RealtimeGeolocation'],
 'credentialSubject': {'id': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
  'geo': {'type': 'GeoCoordinates',
   'latitude': '38.88460',
   'longitude': '-77.02273'}},
 'issuanceDate': '2023-09-16T22:19:21.037587Z'}

Continuing with this scenario, we further imagine that the holder, using the geo-aware digital wallet, now signs and secures the geolocation verifiable credential by generating a JWT token. 

In [16]:
signed_geolocation_vc = sign(KEY_MANAGER, geolocation_vc, "secret3", VERIFIER)
signed_geolocation_vc

'eyJhbGciOiJIUzI1NiIsImtpZCI6InVybjp1dWlkOmU4NGE4ZDFlLWU5YTEtNDc4NS1hMDM4LTVjNWVmZGYxNDk5ZSIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJ1cm46dXVpZDoxOTZhOGM4Zi02ZmQ5LTQ1NmItODE5My00MTU5YjhkYzNhYjYiLCJhdWQiOiJodHRwczovL3ZlcmlmaWVyLmV4YW1wbGUuZ292IiwiZXhwIjoxNjk0OTg5MTYxLCJuYmYiOjE2OTQ5MDI3NjEsImlhdCI6MTY5NDkwMjc2MSwia2lkIjoidXJuOnV1aWQ6ZTg0YThkMWUtZTlhMS00Nzg1LWEwMzgtNWM1ZWZkZjE0OTllIiwic3ViIjoidXJuOnV1aWQ6MTk2YThjOGYtNmZkOS00NTZiLTgxOTMtNDE1OWI4ZGMzYWI2IiwidmMiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjEiLCJodHRwczovL3d3dy53My5vcmcvMjAxOC9jcmVkZW50aWFscy9leGFtcGxlcy92MSIsImh0dHBzOi8vc2NoZW1hLm9yZy8iXSwiaXNzdWVyIjoidXJuOnV1aWQ6MTk2YThjOGYtNmZkOS00NTZiLTgxOTMtNDE1OWI4ZGMzYWI2IiwiaWQiOiJ1cm46dXVpZDplYjViMDgwMC05NjdmLTRkMGItYTM0YS1hZTY3ZWNkYzlhNmIiLCJ0eXBlIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiUmVhbHRpbWVHZW9sb2NhdGlvbiJdLCJjcmVkZW50aWFsU3ViamVjdCI6eyJpZCI6InVybjp1dWlkOjE5NmE4YzhmLTZmZDktNDU2Yi04MTkzLTQxNTliOGRjM2FiNiIsImdlbyI6eyJ0eXBlIjoiR2VvQ29vcmRpbmF0ZXMiLCJsYXRpdHVkZSI6IjM4Lj

### Generating a verifiable presentation combining the geofenced credential with the geolocation credential

The final step in the scenario is for the holder, in the act of providing their geofenced credential from before to a verifier, to combine with it with the geolocation credential generated in the real-tine into a verifiable presentation for presentation to the verifier.

In [17]:
vp_id = urn_uuid(uuid.uuid4())
verifiable_presentation = {
    '@context': [
        'https://www.w3.org/2018/credentials/v1', 
        'https://www.w3.org/2018/credentials/examples/v1', 
        'https://schema.org/'
    ],
    'issuer': subject,
    'id': vp_id,
    'type': ['VerifiablePresentation'],
    'verifiableCredential': [
        signed_geofenced_vc,
        signed_geolocation_vc
    ]
}
verifiable_presentation

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1',
  'https://schema.org/'],
 'issuer': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
 'id': 'urn:uuid:6e8748ec-d791-48ff-912d-0da967113562',
 'type': ['VerifiablePresentation'],
 'verifiableCredential': ['eyJhbGciOiJIUzI1NiIsImtpZCI6InVybjp1dWlkOjdiZmUyMzFhLTM0MWQtNGQ2MS05NDU5LWIwMWE1MTkzNjk1OSIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJ1cm46dXVpZDo5YTVlYjkyYS1iMzRhLTQyYjktOGZiNy0xMGNlOTQ0OWU4M2UiLCJhdWQiOiJodHRwczovL3ZlcmlmaWVyLmV4YW1wbGUuZ292IiwiZXhwIjoxNjk0OTg5MTYxLCJuYmYiOjE2OTQ5MDI3NjEsImlhdCI6MTY5NDkwMjc2MSwia2lkIjoidXJuOnV1aWQ6N2JmZTIzMWEtMzQxZC00ZDYxLTk0NTktYjAxYTUxOTM2OTU5Iiwic3ViIjoidXJuOnV1aWQ6MTk2YThjOGYtNmZkOS00NTZiLTgxOTMtNDE1OWI4ZGMzYWI2IiwidmMiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjIiLCJodHRwczovL3d3dy53My5vcmcvMjAxOC9jcmVkZW50aWFscy9leGFtcGxlcy92MiIsImh0dHBzOi8vc2NoZW1hLm9yZy8iXSwiaXNzdWVyIjoidXJuOnV1aWQ6OWE1ZWI5MmEtYjM0YS00MmI5LThmYjctMTBjZTk0

For good measure, we provide a JSON schema for validating verified presentations, similar to the schema shown above for verified credentials, to ensure that the presentation is syntactically correct.

In [18]:
vp_schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "@context": {
            "type": "array",
            "items": {
                "type": "string"
            },
            "minItems": 1
        },
        "id": {
            "type": "string"
        },
        "type": {
            "type": "array",
            "items": {
                "type": "string"
            },
            "contains": {
                "const": "VerifiablePresentation"
            }
        },
        "issuer": {
            "type": "string"
        },
        "verifiableCredential": {
            "type": "array",
            "items": {
                "type": "string"
            }
        },
        "proof": {
            "type": "object",
            "properties": {
                "type": {
                    "type": "string"
                },
                "created": {
                    "type": "string",
                    "format": "date-time"
                },
                "proofValue": {
                    "type": "string"
                },
                "verificationMethod": {
                    "type": "string"
                },
                "jws": {
                    "type": "string"
                }
            },
            "required": ["type", "created", "proofValue"]
        }
    },
    "required": ["@context", "id", "type", "issuer", "verifiableCredential"]
}

We now show that the presentation is valid according to the schema.

In [19]:
try:
    jsonschema.validate(instance=verifiable_presentation, schema=vp_schema)
    print("Presentation is valid.")
except jsonschema.exceptions.ValidationError as e:
    print("Presentation is not valid:", e)

Presentation is valid.


In [20]:
signed_geoaware_vp = sign(KEY_MANAGER, verifiable_presentation, "secret1", VERIFIER)
signed_geoaware_vp

'eyJhbGciOiJIUzI1NiIsImtpZCI6InVybjp1dWlkOjdmZGU1ZTBlLTc3YWQtNDBiNy1iN2RkLTIwYmYzZWUwZTYzMiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJ1cm46dXVpZDoxOTZhOGM4Zi02ZmQ5LTQ1NmItODE5My00MTU5YjhkYzNhYjYiLCJhdWQiOiJodHRwczovL3ZlcmlmaWVyLmV4YW1wbGUuZ292IiwiZXhwIjoxNjk0OTg5MTYxLCJuYmYiOjE2OTQ5MDI3NjEsImlhdCI6MTY5NDkwMjc2MSwia2lkIjoidXJuOnV1aWQ6N2ZkZTVlMGUtNzdhZC00MGI3LWI3ZGQtMjBiZjNlZTBlNjMyIiwidnAiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjEiLCJodHRwczovL3d3dy53My5vcmcvMjAxOC9jcmVkZW50aWFscy9leGFtcGxlcy92MSIsImh0dHBzOi8vc2NoZW1hLm9yZy8iXSwiaXNzdWVyIjoidXJuOnV1aWQ6MTk2YThjOGYtNmZkOS00NTZiLTgxOTMtNDE1OWI4ZGMzYWI2IiwiaWQiOiJ1cm46dXVpZDo2ZTg3NDhlYy1kNzkxLTQ4ZmYtOTEyZC0wZGE5NjcxMTM1NjIiLCJ0eXBlIjpbIlZlcmlmaWFibGVQcmVzZW50YXRpb24iXSwidmVyaWZpYWJsZUNyZWRlbnRpYWwiOlsiZXlKaGJHY2lPaUpJVXpJMU5pSXNJbXRwWkNJNkluVnlianAxZFdsa09qZGlabVV5TXpGaExUTTBNV1F0TkdRMk1TMDVORFU1TFdJd01XRTFNVGt6TmprMU9TSXNJblI1Y0NJNklrcFhWQ0o5LmV5SnBjM01pT2lKMWNtNDZkWFZwWkRvNVlUVmxZamt5WVMxaU16UmhMVFF5WWprdE9HWmlOeTB4TUdObE

To complete the scenario, we assume that the verifier application, once it has verified and decoded the presentation, is capable of processing the geolocation data in the context of the geofenced credential and is in a position to verify or reject the credential by performing the processing necessary to determine if the geolocation is within the geofence region.

In [21]:
decoded_vp = verify(KEY_MANAGER, signed_geoaware_vp, VERIFIER)
decoded_vp

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1',
  'https://schema.org/'],
 'issuer': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
 'id': 'urn:uuid:6e8748ec-d791-48ff-912d-0da967113562',
 'type': ['VerifiablePresentation'],
 'verifiableCredential': [{'@context': ['https://www.w3.org/2018/credentials/v2',
    'https://www.w3.org/2018/credentials/examples/v2',
    'https://schema.org/'],
   'issuer': 'urn:uuid:9a5eb92a-b34a-42b9-8fb7-10ce9449e83e',
   'id': 'urn:uuid:dc30880d-2d4b-447c-905e-b79075aea95b',
   'type': ['VerifiableCredential', 'Admin merit'],
   'credentialSubject': {'id': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
    'merit': {'First Name': 'Hank',
     'Last Name': 'Lindgren',
     'Email': 'oleg+user2@gomerits.com',
     'Issuing Org Name': 'Merit International, Inc.',
     'Org UUID': 'user2',
     'Admin Phone Number': '6317070926',
     'Org Legal Name': 'Org legal name 74d00472-85d5-4c6b-81d3-5b63

## Summary

In conclusion, with the above proof-of-concept we have provided a technical demonstration of how the Merit platform can be extended to address the interoperability and location awareness requirements defined by the stakeholders at the beginning of this SBIR Phase I effort. As mentioned in the Technical Assessment Report delivered in July 2023, with the current Merit platform's support for the rest of the stakeholder needs described in the project Statement of Work, we believe we have shown that the Merit platform can provide a complete solution for the first responder community. We have described what needs to be addressed in taking this technology demonstrated above and fully building out the capabilities described to a level that would support the delivery of a robust solution as a commercially released and supported product. We close by asserting that we believe that this work shows the significant potential for innovation in verified identity services through leverage of the standards-based approach to credentialing that the Department of Homeland Security is leading.

_This page is governed by the SBIR/STTR Rights legend set forth on the title page, which is incorporated herein, as if written out in full._